# Imports

In [1]:
import os
import logging
import importlib
importlib.reload(logging)
log = logging.getLogger()
log.setLevel('INFO')
import sys

logging.basicConfig(format='%(asctime)s %(levelname)s |: %(message)s',
                     level=logging.INFO, stream=sys.stdout)
import mne
from mne.io import concatenate_raws
import matplotlib.pyplot as plt
from scipy import signal
from sklearn import preprocessing
import numpy as np

import pickle
import json


from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, Activation, Permute, Dropout
from tensorflow.keras.layers import Conv2D, MaxPooling2D, AveragePooling2D
from tensorflow.keras.layers import SeparableConv2D, DepthwiseConv2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import SpatialDropout2D
from tensorflow.keras.regularizers import l1_l2
from tensorflow.keras.layers import Input, Flatten
from tensorflow.keras.constraints import max_norm
from tensorflow.keras import backend as K
from typing import Iterable, Union
import tensorflow as tf
import keras

# Functions

In [36]:
def square(x):
    return K.square(x)

def log(x):
    return K.log(K.clip(x, min_value = 1e-7, max_value = 10000)) 

def create_model(n_classes=2, n_channs=19, samples = 306, dropout_rate=0.5):
    model = Sequential()
    model.add(Conv2D(40, (1, 13), input_shape=(n_channs, samples, 1), kernel_constraint = max_norm(2., axis=(0,1,2))))
    model.add(Conv2D(40, (n_channs, 1), use_bias=False, kernel_constraint = max_norm(2., axis=(0,1,2))))
    model.add(BatchNormalization(epsilon=1e-05, momentum=0.1))
    model.add(Activation(square))
    model.add(AveragePooling2D(pool_size=(1, 35), strides=(1, 7)))
    model.add(Activation(log))
    model.add(Dropout(dropout_rate))
    model.add(Flatten())
    model.add(Dense(n_classes, kernel_constraint = max_norm(0.5), activation='softmax'))
    model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics=['accuracy'])
    return model

def train_model(path='./processed_data/', model_path = './models/', data_type='standardization_one', cuda=True, epochs=15, batch_size=16, multiplier=1, data_type_out=None):
    path = f'{path}{data_type}/'
    
    X_test = np.load(path + f'X_test.npy')
    y_test = np.load(path + f'y_test.npy')
    X_train = np.load(path + f'X_train.npy')
    y_train = np.load(path + f'y_train.npy')
    X_valid = np.load(path + f'X_valid.npy')
    y_valid = np.load(path + f'y_valid.npy')
    
    X_test = (X_test * multiplier).astype(np.float32)
    X_train = (X_train * multiplier).astype(np.float32)
    X_valid = (X_valid * multiplier).astype(np.float32)
    y_test = y_test.astype(np.int64)
    y_train = y_train.astype(np.int64)
    y_valid = y_valid.astype(np.int64)
    
    X = np.concatenate((X_train,X_test))
    y = np.concatenate((y_train,y_test))
    
    y = tf.keras.utils.to_categorical(y, num_classes=2, dtype='float32')
    y_valid = tf.keras.utils.to_categorical(y_valid, num_classes=2, dtype='float32')
    
    X = X.reshape((X.shape[0], X.shape[1], X.shape[2], 1))
    X_valid = X_valid.reshape((X_valid.shape[0], X_valid.shape[1], X_valid.shape[2], 1))
    
    model = create_model()
    history = model.fit(X, y, batch_size = batch_size, epochs = epochs, validation_data=(X_valid, y_valid))
    
    if data_type_out:
        data_type = data_type_out
    model_path = f'{model_path}{data_type}'
    if not os.path.exists(model_path):
        os.makedirs(model_path)
    model.save(model_path)
    history_dict = history.history
    json.dump(history_dict, open(f'{model_path}/history', 'w'))
    
    
def train_model_folds(path='./processed_data/', model_path = './models/', data_type='standardization_one', cuda=True, epochs=15, batch_size=16, multiplier=1, data_type_out=None, folds_num=5):
    path = f'{path}{data_type}/folds/'
    for fold in range(0, folds_num):
        X_test = np.load(path + f'X_test_fold_{fold}.npy')
        y_test = np.load(path + f'y_test_fold_{fold}.npy')
        X_train = np.load(path + f'X_train_fold_{fold}.npy')
        y_train = np.load(path + f'y_train_fold_{fold}.npy')

        X_test = (X_test * multiplier).astype(np.float32)
        X_train = (X_train * multiplier).astype(np.float32)
        y_test = y_test.astype(np.int64)
        y_train = y_train.astype(np.int64)

        y_train = tf.keras.utils.to_categorical(y_train, num_classes=2, dtype='float32')
        y_test = tf.keras.utils.to_categorical(y_test, num_classes=2, dtype='float32')

        X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], X_train.shape[2], 1))
        X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], X_test.shape[2], 1))

        model = create_model()
        history = model.fit(X_train, y_train, batch_size = batch_size, epochs = epochs, validation_data=(X_test, y_test))

        if data_type_out:
            data_type = data_type_out
        model_save_path = f'{model_path}{data_type}/folds/fold_{fold}'
        if not os.path.exists(model_save_path):
            os.makedirs(model_save_path)
        model.save(model_save_path)
        history_dict = history.history
        json.dump(history_dict, open(f'{model_save_path}/history', 'w'))


## Classification clean

In [9]:
train_model(data_type='clean', epochs=50, batch_size=16, multiplier=10**6)

Epoch 1/50
903/903 [==============================] - 24s 26ms/step - loss: 0.8034 - accuracy: 0.5231 - val_loss: 0.7052 - val_accuracy: 0.5487
Epoch 2/50
903/903 [==============================] - 25s 27ms/step - loss: 0.7476 - accuracy: 0.5530 - val_loss: 0.6859 - val_accuracy: 0.5611
Epoch 3/50
903/903 [==============================] - 24s 26ms/step - loss: 0.7460 - accuracy: 0.5663 - val_loss: 0.6957 - val_accuracy: 0.5683
Epoch 4/50
903/903 [==============================] - 24s 26ms/step - loss: 0.7267 - accuracy: 0.5835 - val_loss: 0.7897 - val_accuracy: 0.5429
Epoch 5/50
903/903 [==============================] - 23s 26ms/step - loss: 0.7452 - accuracy: 0.5684 - val_loss: 0.7397 - val_accuracy: 0.5523
Epoch 6/50
903/903 [==============================] - 29s 32ms/step - loss: 0.7424 - accuracy: 0.5661 - val_loss: 0.6935 - val_accuracy: 0.5818
Epoch 7/50
903/903 [==============================] - 26s 29ms/step - loss: 0.7241 - accuracy: 0.5770 - val_loss: 0.7407 - val_accuracy:

## Classification xDawn

In [16]:
train_model(data_type='xDawn', epochs=50, batch_size=16, multiplier=10**6)

Epoch 1/50
903/903 [==============================] - 29s 32ms/step - loss: 0.8166 - accuracy: 0.5206 - val_loss: 0.7130 - val_accuracy: 0.5473
Epoch 2/50
903/903 [==============================] - 26s 29ms/step - loss: 0.7677 - accuracy: 0.5417 - val_loss: 0.8384 - val_accuracy: 0.5101
Epoch 3/50
903/903 [==============================] - 27s 30ms/step - loss: 0.7565 - accuracy: 0.5478 - val_loss: 0.6983 - val_accuracy: 0.5702
Epoch 4/50
903/903 [==============================] - 29s 32ms/step - loss: 0.7605 - accuracy: 0.5384 - val_loss: 0.7155 - val_accuracy: 0.5526
Epoch 5/50
903/903 [==============================] - 29s 32ms/step - loss: 0.7598 - accuracy: 0.5465 - val_loss: 0.7451 - val_accuracy: 0.5550
Epoch 6/50
903/903 [==============================] - 28s 30ms/step - loss: 0.7539 - accuracy: 0.5597 - val_loss: 0.6991 - val_accuracy: 0.5699
Epoch 7/50
903/903 [==============================] - 26s 29ms/step - loss: 0.7526 - accuracy: 0.5538 - val_loss: 0.7024 - val_accuracy:

## Classification standardization common

In [17]:
train_model(data_type='standardization_common', epochs=50, batch_size=16, multiplier=10)

Epoch 1/50
903/903 [==============================] - 23s 26ms/step - loss: 0.7780 - accuracy: 0.5270 - val_loss: 0.7216 - val_accuracy: 0.5382
Epoch 2/50
903/903 [==============================] - 27s 30ms/step - loss: 0.7561 - accuracy: 0.5508 - val_loss: 0.7233 - val_accuracy: 0.5639
Epoch 3/50
903/903 [==============================] - 25s 28ms/step - loss: 0.7353 - accuracy: 0.5631 - val_loss: 0.7103 - val_accuracy: 0.5589
Epoch 4/50
903/903 [==============================] - 25s 28ms/step - loss: 0.7323 - accuracy: 0.5653 - val_loss: 0.6930 - val_accuracy: 0.5823
Epoch 5/50
903/903 [==============================] - 25s 27ms/step - loss: 0.7297 - accuracy: 0.5711 - val_loss: 0.6832 - val_accuracy: 0.5854
Epoch 6/50
903/903 [==============================] - 26s 28ms/step - loss: 0.7137 - accuracy: 0.5905 - val_loss: 0.7019 - val_accuracy: 0.5743
Epoch 7/50
903/903 [==============================] - 30s 33ms/step - loss: 0.7144 - accuracy: 0.5901 - val_loss: 0.6803 - val_accuracy:

## Classification clean participants

In [15]:
train_model(data_type='clean_participants', epochs=50, batch_size=16, multiplier=10**6)

Epoch 1/50
937/937 [==============================] - 30s 31ms/step - loss: 0.7931 - accuracy: 0.5223 - val_loss: 0.6908 - val_accuracy: 0.5631
Epoch 2/50
937/937 [==============================] - 28s 30ms/step - loss: 0.7576 - accuracy: 0.5357 - val_loss: 0.6899 - val_accuracy: 0.5595
Epoch 3/50
937/937 [==============================] - 27s 29ms/step - loss: 0.7477 - accuracy: 0.5501 - val_loss: 0.6836 - val_accuracy: 0.5819
Epoch 4/50
937/937 [==============================] - 30s 32ms/step - loss: 0.7623 - accuracy: 0.5544 - val_loss: 0.7185 - val_accuracy: 0.5881
Epoch 5/50
937/937 [==============================] - 33s 36ms/step - loss: 0.7332 - accuracy: 0.5709 - val_loss: 0.7075 - val_accuracy: 0.5634
Epoch 6/50
937/937 [==============================] - 34s 37ms/step - loss: 0.7286 - accuracy: 0.5725 - val_loss: 0.6947 - val_accuracy: 0.5868
Epoch 7/50
937/937 [==============================] - 32s 34ms/step - loss: 0.7417 - accuracy: 0.5709 - val_loss: 0.6885 - val_accuracy:

## Classification xDawn participants

In [18]:
train_model(data_type='xDawn_participants', epochs=50, batch_size=16, multiplier=10**6)

Epoch 1/50
937/937 [==============================] - 23s 24ms/step - loss: 0.8077 - accuracy: 0.5139 - val_loss: 0.7171 - val_accuracy: 0.5286
Epoch 2/50
937/937 [==============================] - 23s 24ms/step - loss: 0.7701 - accuracy: 0.5337 - val_loss: 0.7273 - val_accuracy: 0.5377
Epoch 3/50
937/937 [==============================] - 23s 24ms/step - loss: 0.7642 - accuracy: 0.5401 - val_loss: 0.7043 - val_accuracy: 0.5608
Epoch 4/50
937/937 [==============================] - 23s 24ms/step - loss: 0.7668 - accuracy: 0.5470 - val_loss: 0.7270 - val_accuracy: 0.5527
Epoch 5/50
937/937 [==============================] - 23s 24ms/step - loss: 0.7536 - accuracy: 0.5404 - val_loss: 0.7701 - val_accuracy: 0.5273
Epoch 6/50
937/937 [==============================] - 23s 24ms/step - loss: 0.7552 - accuracy: 0.5435 - val_loss: 0.6845 - val_accuracy: 0.5748
Epoch 7/50
937/937 [==============================] - 23s 24ms/step - loss: 0.7587 - accuracy: 0.5426 - val_loss: 0.7361 - val_accuracy:

## Classification standardization common participants

In [19]:
train_model(data_type='standardization_common_participants', epochs=50, batch_size=16, multiplier=10)

Epoch 1/50
937/937 [==============================] - 23s 24ms/step - loss: 0.7944 - accuracy: 0.5151 - val_loss: 0.6885 - val_accuracy: 0.5806
Epoch 2/50
937/937 [==============================] - 23s 24ms/step - loss: 0.7610 - accuracy: 0.5432 - val_loss: 0.7144 - val_accuracy: 0.5608
Epoch 3/50
937/937 [==============================] - 23s 24ms/step - loss: 0.7458 - accuracy: 0.5578 - val_loss: 0.7291 - val_accuracy: 0.5702
Epoch 4/50
937/937 [==============================] - 23s 24ms/step - loss: 0.7327 - accuracy: 0.5647 - val_loss: 0.7391 - val_accuracy: 0.5549
Epoch 5/50
937/937 [==============================] - 23s 24ms/step - loss: 0.7238 - accuracy: 0.5764 - val_loss: 0.6971 - val_accuracy: 0.5839
Epoch 6/50
937/937 [==============================] - 22s 24ms/step - loss: 0.7201 - accuracy: 0.5870 - val_loss: 0.6915 - val_accuracy: 0.5816
Epoch 7/50
937/937 [==============================] - 23s 24ms/step - loss: 0.7098 - accuracy: 0.5922 - val_loss: 0.6810 - val_accuracy:

# K-fold

## Classification clean participants K-fold

In [ ]:
train_model_folds(data_type='clean_participants', epochs=50, batch_size=16, multiplier=10**6, folds_num=5)

Epoch 1/50
740/740 [==============================] - 19s 25ms/step - loss: 0.7974 - accuracy: 0.5159 - val_loss: 0.7147 - val_accuracy: 0.5291
Epoch 2/50
740/740 [==============================] - 19s 25ms/step - loss: 0.7714 - accuracy: 0.5409 - val_loss: 0.7362 - val_accuracy: 0.5253
Epoch 3/50
740/740 [==============================] - 19s 25ms/step - loss: 0.7444 - accuracy: 0.5632 - val_loss: 0.7215 - val_accuracy: 0.5538
Epoch 4/50
740/740 [==============================] - 18s 25ms/step - loss: 0.7364 - accuracy: 0.5726 - val_loss: 0.7440 - val_accuracy: 0.5386
Epoch 5/50
740/740 [==============================] - ETA: 0s - loss: 0.7251 - accuracy: 0.5792

## Classification xDawn participants K-fold

In [46]:
train_model_folds(data_type='xDawn_participants', epochs=50, batch_size=16, multiplier=10**6, folds_num=5)

Epoch 1/50
740/740 [==============================] - 19s 25ms/step - loss: 0.8156 - accuracy: 0.5133 - val_loss: 0.7193 - val_accuracy: 0.5234
Epoch 2/50
740/740 [==============================] - 19s 25ms/step - loss: 0.7684 - accuracy: 0.5495 - val_loss: 0.7488 - val_accuracy: 0.5120
Epoch 3/50
740/740 [==============================] - 19s 25ms/step - loss: 0.7600 - accuracy: 0.5479 - val_loss: 0.8268 - val_accuracy: 0.5155
Epoch 4/50
740/740 [==============================] - 19s 25ms/step - loss: 0.7732 - accuracy: 0.5397 - val_loss: 0.7857 - val_accuracy: 0.5120
Epoch 5/50
740/740 [==============================] - 18s 25ms/step - loss: 0.7577 - accuracy: 0.5523 - val_loss: 0.7529 - val_accuracy: 0.5386
Epoch 6/50
740/740 [==============================] - 18s 25ms/step - loss: 0.7612 - accuracy: 0.5523 - val_loss: 0.7673 - val_accuracy: 0.5291
Epoch 7/50
740/740 [==============================] - 19s 25ms/step - loss: 0.7612 - accuracy: 0.5482 - val_loss: 0.7255 - val_accuracy:

Epoch 7/50
757/757 [==============================] - 19s 25ms/step - loss: 0.7574 - accuracy: 0.5539 - val_loss: 0.7110 - val_accuracy: 0.5570
Epoch 8/50
757/757 [==============================] - 19s 25ms/step - loss: 0.7571 - accuracy: 0.5410 - val_loss: 0.7067 - val_accuracy: 0.5820
Epoch 9/50
757/757 [==============================] - 19s 25ms/step - loss: 0.7581 - accuracy: 0.5338 - val_loss: 0.7550 - val_accuracy: 0.5372
Epoch 10/50
757/757 [==============================] - 19s 25ms/step - loss: 0.7540 - accuracy: 0.5424 - val_loss: 0.7500 - val_accuracy: 0.5410
Epoch 11/50
757/757 [==============================] - 19s 25ms/step - loss: 0.7548 - accuracy: 0.5401 - val_loss: 0.7404 - val_accuracy: 0.5657
Epoch 12/50
757/757 [==============================] - 19s 25ms/step - loss: 0.7492 - accuracy: 0.5494 - val_loss: 0.8251 - val_accuracy: 0.5382
Epoch 13/50
757/757 [==============================] - 19s 25ms/step - loss: 0.7523 - accuracy: 0.5515 - val_loss: 0.7037 - val_accur

Epoch 38/50
752/752 [==============================] - 19s 25ms/step - loss: 0.6779 - accuracy: 0.5837 - val_loss: 0.6861 - val_accuracy: 0.5596
Epoch 39/50
752/752 [==============================] - 19s 25ms/step - loss: 0.6704 - accuracy: 0.5922 - val_loss: 0.6935 - val_accuracy: 0.5482
Epoch 40/50
752/752 [==============================] - 19s 25ms/step - loss: 0.6824 - accuracy: 0.5749 - val_loss: 0.7010 - val_accuracy: 0.5489
Epoch 41/50
752/752 [==============================] - 19s 25ms/step - loss: 0.6773 - accuracy: 0.5856 - val_loss: 0.6818 - val_accuracy: 0.5677
Epoch 42/50
752/752 [==============================] - 19s 25ms/step - loss: 0.6715 - accuracy: 0.5910 - val_loss: 0.6785 - val_accuracy: 0.5772
Epoch 43/50
752/752 [==============================] - 19s 25ms/step - loss: 0.6724 - accuracy: 0.5941 - val_loss: 0.6784 - val_accuracy: 0.5644
Epoch 44/50
752/752 [==============================] - 19s 25ms/step - loss: 0.6732 - accuracy: 0.5878 - val_loss: 0.6767 - val_ac

## Classification standardization common participants K-fold

In [ ]:
train_model_folds(data_type='standardization_common_participants', epochs=50, batch_size=16, multiplier=10, folds_num=5)

Epoch 1/50
740/740 [==============================] - 19s 25ms/step - loss: 0.7703 - accuracy: 0.5354 - val_loss: 0.7184 - val_accuracy: 0.5452
Epoch 2/50
740/740 [==============================] - 19s 25ms/step - loss: 0.7471 - accuracy: 0.5502 - val_loss: 0.7425 - val_accuracy: 0.5361
Epoch 3/50
740/740 [==============================] - 19s 25ms/step - loss: 0.7332 - accuracy: 0.5725 - val_loss: 0.7498 - val_accuracy: 0.5408
Epoch 4/50
740/740 [==============================] - 18s 25ms/step - loss: 0.7319 - accuracy: 0.5719 - val_loss: 0.9328 - val_accuracy: 0.5307
Epoch 5/50
740/740 [==============================] - 19s 25ms/step - loss: 0.7205 - accuracy: 0.5832 - val_loss: 0.7301 - val_accuracy: 0.5528
Epoch 6/50
740/740 [==============================] - 19s 25ms/step - loss: 0.7157 - accuracy: 0.5875 - val_loss: 0.7782 - val_accuracy: 0.5506
Epoch 7/50
740/740 [==============================] - 19s 25ms/step - loss: 0.7035 - accuracy: 0.5985 - val_loss: 0.9495 - val_accuracy:

Epoch 7/50
757/757 [==============================] - 19s 25ms/step - loss: 0.7224 - accuracy: 0.5820 - val_loss: 0.6714 - val_accuracy: 0.6074
Epoch 8/50
757/757 [==============================] - 19s 25ms/step - loss: 0.7196 - accuracy: 0.5876 - val_loss: 0.6750 - val_accuracy: 0.6042
Epoch 9/50
757/757 [==============================] - 19s 25ms/step - loss: 0.7275 - accuracy: 0.5799 - val_loss: 0.6982 - val_accuracy: 0.5994
Epoch 10/50
757/757 [==============================] - 19s 25ms/step - loss: 0.7108 - accuracy: 0.5930 - val_loss: 0.7023 - val_accuracy: 0.5949
Epoch 11/50
757/757 [==============================] - 19s 25ms/step - loss: 0.7083 - accuracy: 0.5950 - val_loss: 0.6842 - val_accuracy: 0.6056
Epoch 12/50
757/757 [==============================] - 19s 25ms/step - loss: 0.6913 - accuracy: 0.6104 - val_loss: 0.6914 - val_accuracy: 0.5997
Epoch 13/50
757/757 [==============================] - 19s 25ms/step - loss: 0.7105 - accuracy: 0.5918 - val_loss: 0.6791 - val_accur

Epoch 13/50
739/739 [==============================] - 19s 25ms/step - loss: 0.6827 - accuracy: 0.6077 - val_loss: 0.7057 - val_accuracy: 0.5885
Epoch 14/50
739/739 [==============================] - 19s 25ms/step - loss: 0.6969 - accuracy: 0.6026 - val_loss: 0.7127 - val_accuracy: 0.5667
Epoch 15/50
739/739 [==============================] - 19s 25ms/step - loss: 0.6917 - accuracy: 0.6099 - val_loss: 0.7292 - val_accuracy: 0.5452
Epoch 16/50
739/739 [==============================] - 19s 25ms/step - loss: 0.6727 - accuracy: 0.6166 - val_loss: 0.7489 - val_accuracy: 0.5765
Epoch 17/50
739/739 [==============================] - 19s 25ms/step - loss: 0.6717 - accuracy: 0.6163 - val_loss: 0.7355 - val_accuracy: 0.5626
Epoch 18/50
739/739 [==============================] - 19s 25ms/step - loss: 0.6734 - accuracy: 0.6183 - val_loss: 0.7489 - val_accuracy: 0.5629
Epoch 19/50
739/739 [==============================] - 19s 25ms/step - loss: 0.6667 - accuracy: 0.6243 - val_loss: 0.7225 - val_ac

Epoch 19/50
761/761 [==============================] - 19s 25ms/step - loss: 0.6770 - accuracy: 0.6103 - val_loss: 0.6978 - val_accuracy: 0.5861
Epoch 20/50
761/761 [==============================] - 19s 25ms/step - loss: 0.6713 - accuracy: 0.6258 - val_loss: 0.7431 - val_accuracy: 0.5620
Epoch 21/50
761/761 [==============================] - 19s 25ms/step - loss: 0.6799 - accuracy: 0.6197 - val_loss: 0.6849 - val_accuracy: 0.6011
Epoch 22/50
761/761 [==============================] - 19s 25ms/step - loss: 0.6782 - accuracy: 0.6209 - val_loss: 0.7006 - val_accuracy: 0.5922
Epoch 23/50
761/761 [==============================] - 19s 25ms/step - loss: 0.6724 - accuracy: 0.6181 - val_loss: 0.6941 - val_accuracy: 0.5940
Epoch 24/50
761/761 [==============================] - 19s 25ms/step - loss: 0.6722 - accuracy: 0.6173 - val_loss: 0.7142 - val_accuracy: 0.6036
Epoch 25/50
761/761 [==============================] - 19s 25ms/step - loss: 0.6709 - accuracy: 0.6263 - val_loss: 0.7152 - val_ac

Epoch 25/50
752/752 [==============================] - 19s 25ms/step - loss: 0.6651 - accuracy: 0.6289 - val_loss: 0.7205 - val_accuracy: 0.5829
Epoch 26/50
752/752 [==============================] - 19s 25ms/step - loss: 0.6595 - accuracy: 0.6298 - val_loss: 0.7043 - val_accuracy: 0.5940
Epoch 27/50
752/752 [==============================] - 19s 25ms/step - loss: 0.6701 - accuracy: 0.6256 - val_loss: 0.7078 - val_accuracy: 0.5920
Epoch 28/50
752/752 [==============================] - 19s 25ms/step - loss: 0.6538 - accuracy: 0.6340 - val_loss: 0.7045 - val_accuracy: 0.5772
Epoch 29/50
752/752 [==============================] - 19s 25ms/step - loss: 0.6633 - accuracy: 0.6255 - val_loss: 0.7190 - val_accuracy: 0.5916
Epoch 30/50
752/752 [==============================] - 19s 25ms/step - loss: 0.6547 - accuracy: 0.6402 - val_loss: 0.7529 - val_accuracy: 0.5455
Epoch 31/50
752/752 [==============================] - 19s 25ms/step - loss: 0.6461 - accuracy: 0.6464 - val_loss: 0.7470 - val_ac

## Classification clean K-fold

In [50]:
train_model_folds(data_type='clean', epochs=50, batch_size=16, multiplier=10**6, folds_num=5)

Epoch 1/50
769/769 [==============================] - 22s 26ms/step - loss: 0.7976 - accuracy: 0.5153 - val_loss: 0.7162 - val_accuracy: 0.5532
Epoch 2/50
769/769 [==============================] - 19s 25ms/step - loss: 0.7527 - accuracy: 0.5442 - val_loss: 0.7412 - val_accuracy: 0.5490
Epoch 3/50
769/769 [==============================] - 19s 25ms/step - loss: 0.7401 - accuracy: 0.5614 - val_loss: 0.7149 - val_accuracy: 0.5577
Epoch 4/50
769/769 [==============================] - 19s 25ms/step - loss: 0.7457 - accuracy: 0.5579 - val_loss: 0.7792 - val_accuracy: 0.5490
Epoch 5/50
769/769 [==============================] - 19s 25ms/step - loss: 0.7320 - accuracy: 0.5679 - val_loss: 0.6851 - val_accuracy: 0.5810
Epoch 6/50
769/769 [==============================] - 19s 25ms/step - loss: 0.7366 - accuracy: 0.5604 - val_loss: 0.6849 - val_accuracy: 0.5846
Epoch 7/50
769/769 [==============================] - 19s 25ms/step - loss: 0.7300 - accuracy: 0.5837 - val_loss: 0.6846 - val_accuracy:

769/769 [==============================] - 23s 29ms/step - loss: 0.7183 - accuracy: 0.5867 - val_loss: 0.7068 - val_accuracy: 0.5655
Epoch 8/50
769/769 [==============================] - 23s 29ms/step - loss: 0.7350 - accuracy: 0.5802 - val_loss: 0.7039 - val_accuracy: 0.5759
Epoch 9/50
769/769 [==============================] - 23s 29ms/step - loss: 0.7186 - accuracy: 0.5875 - val_loss: 0.6873 - val_accuracy: 0.5898
Epoch 10/50
769/769 [==============================] - 22s 29ms/step - loss: 0.7038 - accuracy: 0.6077 - val_loss: 0.6640 - val_accuracy: 0.6145
Epoch 11/50
769/769 [==============================] - 22s 29ms/step - loss: 0.6991 - accuracy: 0.5996 - val_loss: 0.6986 - val_accuracy: 0.5762
Epoch 12/50
769/769 [==============================] - 22s 29ms/step - loss: 0.7164 - accuracy: 0.5894 - val_loss: 0.7063 - val_accuracy: 0.5885
Epoch 13/50
769/769 [==============================] - 22s 29ms/step - loss: 0.6938 - accuracy: 0.6134 - val_loss: 0.6706 - val_accuracy: 0.6028

770/770 [==============================] - 19s 25ms/step - loss: 0.6666 - accuracy: 0.6246 - val_loss: 0.6796 - val_accuracy: 0.6043
Epoch 27/50
770/770 [==============================] - 19s 25ms/step - loss: 0.6654 - accuracy: 0.6233 - val_loss: 0.6873 - val_accuracy: 0.5978
Epoch 28/50
770/770 [==============================] - 19s 25ms/step - loss: 0.6678 - accuracy: 0.6257 - val_loss: 0.7019 - val_accuracy: 0.5897
Epoch 29/50
770/770 [==============================] - 19s 25ms/step - loss: 0.6502 - accuracy: 0.6359 - val_loss: 0.7010 - val_accuracy: 0.5936
Epoch 30/50
770/770 [==============================] - 20s 25ms/step - loss: 0.6614 - accuracy: 0.6280 - val_loss: 0.6934 - val_accuracy: 0.5893
Epoch 31/50
770/770 [==============================] - 19s 25ms/step - loss: 0.6521 - accuracy: 0.6323 - val_loss: 0.7053 - val_accuracy: 0.5874
Epoch 32/50
770/770 [==============================] - 19s 25ms/step - loss: 0.6640 - accuracy: 0.6181 - val_loss: 0.6714 - val_accuracy: 0.60

770/770 [==============================] - 19s 25ms/step - loss: 0.6675 - accuracy: 0.6171 - val_loss: 0.7047 - val_accuracy: 0.5964
Epoch 33/50
770/770 [==============================] - 19s 25ms/step - loss: 0.6604 - accuracy: 0.6243 - val_loss: 0.7083 - val_accuracy: 0.6077
Epoch 34/50
770/770 [==============================] - 19s 25ms/step - loss: 0.6587 - accuracy: 0.6265 - val_loss: 0.6870 - val_accuracy: 0.5807
Epoch 35/50
770/770 [==============================] - 19s 25ms/step - loss: 0.6490 - accuracy: 0.6259 - val_loss: 0.6833 - val_accuracy: 0.5918
Epoch 36/50
770/770 [==============================] - 19s 25ms/step - loss: 0.6436 - accuracy: 0.6440 - val_loss: 0.7286 - val_accuracy: 0.5807
Epoch 37/50
770/770 [==============================] - 19s 25ms/step - loss: 0.6479 - accuracy: 0.6373 - val_loss: 0.6749 - val_accuracy: 0.6051
Epoch 38/50
770/770 [==============================] - 19s 25ms/step - loss: 0.6502 - accuracy: 0.6373 - val_loss: 0.7118 - val_accuracy: 0.57

771/771 [==============================] - 19s 25ms/step - loss: 0.6600 - accuracy: 0.6194 - val_loss: 0.6790 - val_accuracy: 0.5988
Epoch 39/50
771/771 [==============================] - 19s 25ms/step - loss: 0.6597 - accuracy: 0.6326 - val_loss: 0.6627 - val_accuracy: 0.6105
Epoch 40/50
771/771 [==============================] - 19s 25ms/step - loss: 0.6461 - accuracy: 0.6364 - val_loss: 0.7074 - val_accuracy: 0.5762
Epoch 41/50
771/771 [==============================] - 19s 25ms/step - loss: 0.6572 - accuracy: 0.6273 - val_loss: 0.6969 - val_accuracy: 0.5811
Epoch 42/50
771/771 [==============================] - 19s 25ms/step - loss: 0.6639 - accuracy: 0.6213 - val_loss: 0.6560 - val_accuracy: 0.6121
Epoch 43/50
771/771 [==============================] - 19s 25ms/step - loss: 0.6462 - accuracy: 0.6362 - val_loss: 0.6752 - val_accuracy: 0.5958
Epoch 44/50
771/771 [==============================] - 19s 25ms/step - loss: 0.6423 - accuracy: 0.6434 - val_loss: 0.6812 - val_accuracy: 0.59

## Classification xDawn K-fold

In [ ]:
train_model_folds(data_type='xDawn', epochs=50, batch_size=16, multiplier=10**6, folds_num=5)

Epoch 1/50
769/769 [==============================] - 20s 26ms/step - loss: 0.8060 - accuracy: 0.5236 - val_loss: 0.7202 - val_accuracy: 0.5134
Epoch 2/50
769/769 [==============================] - 19s 25ms/step - loss: 0.7700 - accuracy: 0.5277 - val_loss: 0.7210 - val_accuracy: 0.5309
Epoch 3/50
769/769 [==============================] - 19s 25ms/step - loss: 0.7670 - accuracy: 0.5363 - val_loss: 0.7006 - val_accuracy: 0.5581
Epoch 4/50
769/769 [==============================] - 19s 25ms/step - loss: 0.7525 - accuracy: 0.5484 - val_loss: 0.7405 - val_accuracy: 0.5377
Epoch 5/50
769/769 [==============================] - 19s 25ms/step - loss: 0.7535 - accuracy: 0.5553 - val_loss: 0.7111 - val_accuracy: 0.5480
Epoch 6/50
769/769 [==============================] - 19s 25ms/step - loss: 0.7568 - accuracy: 0.5414 - val_loss: 0.7362 - val_accuracy: 0.5147
Epoch 7/50
769/769 [==============================] - 19s 25ms/step - loss: 0.7511 - accuracy: 0.5491 - val_loss: 0.7065 - val_accuracy:

770/770 [==============================] - 19s 25ms/step - loss: 0.7399 - accuracy: 0.5465 - val_loss: 0.6986 - val_accuracy: 0.5724
Epoch 16/50
770/770 [==============================] - 19s 25ms/step - loss: 0.7332 - accuracy: 0.5613 - val_loss: 0.7303 - val_accuracy: 0.5403
Epoch 17/50
770/770 [==============================] - 19s 25ms/step - loss: 0.7442 - accuracy: 0.5511 - val_loss: 0.7065 - val_accuracy: 0.5572
Epoch 18/50
770/770 [==============================] - 19s 25ms/step - loss: 0.7351 - accuracy: 0.5575 - val_loss: 0.7286 - val_accuracy: 0.5526
Epoch 19/50
770/770 [==============================] - 19s 25ms/step - loss: 0.7306 - accuracy: 0.5614 - val_loss: 0.6837 - val_accuracy: 0.5637
Epoch 20/50
770/770 [==============================] - 19s 25ms/step - loss: 0.7232 - accuracy: 0.5596 - val_loss: 0.6829 - val_accuracy: 0.5796
Epoch 21/50
770/770 [==============================] - 19s 25ms/step - loss: 0.7180 - accuracy: 0.5622 - val_loss: 0.7343 - val_accuracy: 0.56

770/770 [==============================] - 19s 25ms/step - loss: 0.7144 - accuracy: 0.5583 - val_loss: 0.7198 - val_accuracy: 0.5426
Epoch 22/50
770/770 [==============================] - 19s 25ms/step - loss: 0.7098 - accuracy: 0.5698 - val_loss: 0.6888 - val_accuracy: 0.5618
Epoch 23/50
770/770 [==============================] - 19s 25ms/step - loss: 0.7026 - accuracy: 0.5753 - val_loss: 0.7023 - val_accuracy: 0.5439
Epoch 24/50
770/770 [==============================] - 19s 25ms/step - loss: 0.6990 - accuracy: 0.5653 - val_loss: 0.6854 - val_accuracy: 0.5703
Epoch 25/50
770/770 [==============================] - 19s 25ms/step - loss: 0.7003 - accuracy: 0.5634 - val_loss: 0.7010 - val_accuracy: 0.5605
Epoch 26/50
770/770 [==============================] - 19s 25ms/step - loss: 0.6958 - accuracy: 0.5740 - val_loss: 0.6925 - val_accuracy: 0.5586
Epoch 27/50
770/770 [==============================] - 19s 25ms/step - loss: 0.6907 - accuracy: 0.5690 - val_loss: 0.6978 - val_accuracy: 0.56

771/771 [==============================] - 19s 25ms/step - loss: 0.6987 - accuracy: 0.5692 - val_loss: 0.6854 - val_accuracy: 0.5873
Epoch 28/50
771/771 [==============================] - 19s 25ms/step - loss: 0.6989 - accuracy: 0.5709 - val_loss: 0.7049 - val_accuracy: 0.5439
Epoch 29/50
771/771 [==============================] - 20s 26ms/step - loss: 0.6964 - accuracy: 0.5634 - val_loss: 0.6817 - val_accuracy: 0.5697
Epoch 30/50
771/771 [==============================] - 20s 25ms/step - loss: 0.6985 - accuracy: 0.5707 - val_loss: 0.6839 - val_accuracy: 0.5671
Epoch 31/50
771/771 [==============================] - 19s 25ms/step - loss: 0.6907 - accuracy: 0.5730 - val_loss: 0.6858 - val_accuracy: 0.5475
Epoch 32/50
771/771 [==============================] - 19s 25ms/step - loss: 0.6929 - accuracy: 0.5657 - val_loss: 0.6846 - val_accuracy: 0.5615
Epoch 33/50
771/771 [==============================] - 19s 25ms/step - loss: 0.6917 - accuracy: 0.5690 - val_loss: 0.6790 - val_accuracy: 0.57

## Classification standardization common K-fold

In [ ]:
train_model_folds(data_type='standardization_common', epochs=50, batch_size=16, multiplier=10, folds_num=5)

Epoch 1/50
769/769 [==============================] - 20s 26ms/step - loss: 0.7740 - accuracy: 0.5306 - val_loss: 0.6990 - val_accuracy: 0.5539
Epoch 2/50
769/769 [==============================] - 19s 25ms/step - loss: 0.7471 - accuracy: 0.5493 - val_loss: 0.7454 - val_accuracy: 0.5448
Epoch 3/50
769/769 [==============================] - 19s 25ms/step - loss: 0.7416 - accuracy: 0.5665 - val_loss: 0.7306 - val_accuracy: 0.5451
Epoch 4/50
769/769 [==============================] - 19s 25ms/step - loss: 0.7377 - accuracy: 0.5619 - val_loss: 0.7052 - val_accuracy: 0.5668
Epoch 5/50
769/769 [==============================] - 19s 25ms/step - loss: 0.7274 - accuracy: 0.5800 - val_loss: 0.6953 - val_accuracy: 0.5814
Epoch 6/50
769/769 [==============================] - 19s 25ms/step - loss: 0.7084 - accuracy: 0.5939 - val_loss: 0.7086 - val_accuracy: 0.5694
Epoch 7/50
769/769 [==============================] - 19s 25ms/step - loss: 0.7050 - accuracy: 0.6005 - val_loss: 0.7189 - val_accuracy:

Epoch 7/50
769/769 [==============================] - 19s 25ms/step - loss: 0.7243 - accuracy: 0.5753 - val_loss: 0.6913 - val_accuracy: 0.5905
Epoch 8/50
769/769 [==============================] - 19s 25ms/step - loss: 0.7158 - accuracy: 0.5830 - val_loss: 0.7084 - val_accuracy: 0.5908
Epoch 9/50
769/769 [==============================] - 19s 25ms/step - loss: 0.7112 - accuracy: 0.5913 - val_loss: 0.7001 - val_accuracy: 0.5707
Epoch 10/50
769/769 [==============================] - 19s 25ms/step - loss: 0.7065 - accuracy: 0.5940 - val_loss: 0.6944 - val_accuracy: 0.5846
Epoch 11/50
769/769 [==============================] - 19s 25ms/step - loss: 0.6954 - accuracy: 0.6012 - val_loss: 0.6891 - val_accuracy: 0.5856
Epoch 12/50
769/769 [==============================] - 20s 25ms/step - loss: 0.7061 - accuracy: 0.5901 - val_loss: 0.6769 - val_accuracy: 0.6119
Epoch 13/50
769/769 [==============================] - 19s 25ms/step - loss: 0.6946 - accuracy: 0.6034 - val_loss: 0.6923 - val_accur

Epoch 13/50
770/770 [==============================] - 19s 25ms/step - loss: 0.6949 - accuracy: 0.6078 - val_loss: 0.6960 - val_accuracy: 0.5906
Epoch 14/50
770/770 [==============================] - 19s 25ms/step - loss: 0.6844 - accuracy: 0.6173 - val_loss: 0.6913 - val_accuracy: 0.6079
Epoch 15/50
770/770 [==============================] - 19s 25ms/step - loss: 0.6903 - accuracy: 0.6101 - val_loss: 0.6885 - val_accuracy: 0.5910
Epoch 16/50
770/770 [==============================] - 19s 25ms/step - loss: 0.6820 - accuracy: 0.6177 - val_loss: 0.6799 - val_accuracy: 0.6020
Epoch 17/50
770/770 [==============================] - 19s 25ms/step - loss: 0.6691 - accuracy: 0.6261 - val_loss: 0.7170 - val_accuracy: 0.5676
Epoch 18/50
770/770 [==============================] - 19s 25ms/step - loss: 0.6698 - accuracy: 0.6200 - val_loss: 0.6787 - val_accuracy: 0.6033
Epoch 19/50
770/770 [==============================] - 19s 25ms/step - loss: 0.6748 - accuracy: 0.6211 - val_loss: 0.6779 - val_ac

Epoch 19/50
770/770 [==============================] - 19s 25ms/step - loss: 0.6759 - accuracy: 0.6216 - val_loss: 0.6923 - val_accuracy: 0.5885
Epoch 20/50
770/770 [==============================] - 19s 25ms/step - loss: 0.6802 - accuracy: 0.6134 - val_loss: 0.7078 - val_accuracy: 0.5898
Epoch 21/50
770/770 [==============================] - 19s 25ms/step - loss: 0.6650 - accuracy: 0.6288 - val_loss: 0.6865 - val_accuracy: 0.6003
Epoch 22/50
770/770 [==============================] - 19s 25ms/step - loss: 0.6670 - accuracy: 0.6285 - val_loss: 0.6744 - val_accuracy: 0.6058
Epoch 23/50
770/770 [==============================] - 19s 25ms/step - loss: 0.6716 - accuracy: 0.6134 - val_loss: 0.6933 - val_accuracy: 0.6051
Epoch 24/50
770/770 [==============================] - 19s 25ms/step - loss: 0.6685 - accuracy: 0.6195 - val_loss: 0.6923 - val_accuracy: 0.5928
Epoch 25/50
770/770 [==============================] - 19s 25ms/step - loss: 0.6679 - accuracy: 0.6252 - val_loss: 0.6822 - val_ac